In [1]:
import pandas as pd

pretest_data_path = "./data/pretest.xlsx"
pretest_data = pd.read_excel(pretest_data_path)

pretest_data

,conversations,Blame,error
0,Q: 我问过你们线下的售后 说是可以装\nA1: 移位了不能安装我们虹吸马桶哦亲，会造成堵塞...,NaN,NaN
1,A1: 亲亲，这里是客服新德里，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价...,NaN,NaN
2,Q: 给我按这个推荐一款\nA1: 亲亲，您好，在的呢。这里是售前客服荷兰，很荣幸为您服务！...,NaN,NaN
3,A1: 亲亲，您好，在的呢。这里是售前客服丽江，很荣幸为您服务！​​​\nA0: · 稍等，...,NaN,NaN
4,A1: 亲亲，这里是客服柏林，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价】...,NaN,NaN
5,Q: 拍下后可延迟发货吗\nA1: 亲亲，您好，在的呢。这里是售前客服丽江，很荣幸为您服务！...,NaN,NaN
6,Q: 在吗\nA1: 在的哦，亲亲\nQ: 大单槽\nQ: 目前的盆是80。45的\nQ: ...,NaN,NaN
7,A1: 亲亲，您好，在的呢。这里是售前客服京都，很荣幸为您服务！​​​\nA1: 在的哦亲亲...,NaN,NaN
8,A1: 亲亲，这里是客服新德里，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价...,NaN,NaN
9,A1: 亲亲，这里是客服香格里拉，很高兴为您服务~----💥6.18狂欢节，疯抢中~~活动时...,NaN,NaN


In [2]:
from dotenv import load_dotenv
import os


load_dotenv()
openai_api_key = os.getenv("OPENROUTER_API_KEY")
openai_api_base = "https://openrouter.ai/api/v1"

In [3]:
llm_model = "qwen/qwen3-235b-a22b:free"
# llm_model = "qwen/qwen3-32b:free"
model_name = "qwen"
# llm_model = "deepseek/deepseek-v3-base:free"
# model_name = "deepseek"
# llm_model = "microsoft/phi-4-reasoning:free"
# model_name = 'phi4reason'
# llm_model = "nousresearch/deephermes-3-mistral-24b-preview:free"
# model_name = "deephermes"

In [4]:
blame_prompt = """
你是一个数据标注师，
【场景描述】
本次标注的数据来自电商平台的客服咨询对话场景。在该场景中：
用户发起咨询（Q）
同时有AI客服（A0）和人工客服（A1）参与回答
对话按时间顺序记录

【数据格式】
每条对话记录格式如下：
Q：[用户消息内容]
A0：[AI客服回复内容]
A1：[人工客服回复内容]
(按实际时间顺序排列，可能交替出现)

【标注任务】
请以第三方视角阅读对话记录，判断是否存在以下情况：

工客服推责行为（blame）：
判断标准：人工客服（A1）是否存在将责任推给AI客服（A0）的表述
示例（存在推责行为）：
"刚才那是AI自动回复的，可能不准确"
"AI客服之前的回答不用理会"
"系统自动发送了错误信息"

请仅回答True或False，不要解释
"""

error_prompt = """
你是一个数据标注师，
【场景描述】
本次标注的数据来自电商平台的客服咨询对话场景。在该场景中：
用户发起咨询（Q）
同时有AI客服（A0）和人工客服（A1）参与回答
对话按时间顺序记录

【数据格式】
每条对话记录格式如下：
Q：[用户消息内容]
A0：[AI客服回复内容]
A1：[人工客服回复内容]
(按实际时间顺序排列，可能交替出现)

【标注任务】
请以第三方视角阅读对话记录，判断是否存在以下情况：

客服回答错误
判断标准：客服（A0或A1）是否存在回答用户问题出错/不准确的情况，通常表现为
1. 用户后续追问或质疑
2. 客服后续纠正自己之前的回答
3. 明显的事实性错误


请仅回答True或False，不要解释
"""

In [5]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
import time

model = init_chat_model(
    model=llm_model,
    model_provider="openai",
    openai_api_base=openai_api_base,
    openai_api_key=openai_api_key,
)


In [6]:
def invoke_with_retry(model, messages, max_retries, retry_delay):
    for attempt in range(max_retries):
        try:
            return model.invoke(messages)
        except Exception as e:
            print(
                f"Attempt {attempt + 1} failed: {e}. Retrying in {retry_delay} seconds..."
            )
            time.sleep(retry_delay)
    raise Exception("Max retries exceeded.")

In [7]:
max_retries = 20
retry_delay = 5  # seconds

conversations = pretest_data["conversations"]
results = []

for conv in conversations:
    blame_messages = [
        SystemMessage(blame_prompt),
        HumanMessage(conv),
    ]
    error_messages = [
        SystemMessage(error_prompt),
        HumanMessage(conv),
    ]
    
    blame_res = invoke_with_retry(model, blame_messages, max_retries, retry_delay)
    error_res = invoke_with_retry(model, error_messages, max_retries, retry_delay)
    
    blame_flag = 1 if blame_res.content == 'True' else 0
    error_flag = 1 if error_res.content == 'True' else 0
    print('----------------------------------')
    # print("Original Conversation:\n", conv, sep='')
    print("Blame:", blame_flag)
    print("Error:", error_flag)

    results.append({
        "Conversation": conv,
        "Blame": blame_flag,
        "Error": error_flag
    })

df = pd.DataFrame(results)
df

----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 1


,Conversation,Blame,Error
0,Q: 我问过你们线下的售后 说是可以装\nA1: 移位了不能安装我们虹吸马桶哦亲，会造成堵塞...,0,0
1,A1: 亲亲，这里是客服新德里，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价...,0,1
2,Q: 给我按这个推荐一款\nA1: 亲亲，您好，在的呢。这里是售前客服荷兰，很荣幸为您服务！...,0,1
3,A1: 亲亲，您好，在的呢。这里是售前客服丽江，很荣幸为您服务！​​​\nA0: · 稍等，...,1,1
4,A1: 亲亲，这里是客服柏林，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价】...,0,0
5,Q: 拍下后可延迟发货吗\nA1: 亲亲，您好，在的呢。这里是售前客服丽江，很荣幸为您服务！...,1,1
6,Q: 在吗\nA1: 在的哦，亲亲\nQ: 大单槽\nQ: 目前的盆是80。45的\nQ: ...,0,1
7,A1: 亲亲，您好，在的呢。这里是售前客服京都，很荣幸为您服务！​​​\nA1: 在的哦亲亲...,0,1
8,A1: 亲亲，这里是客服新德里，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价...,1,1
9,A1: 亲亲，这里是客服香格里拉，很高兴为您服务~----💥6.18狂欢节，疯抢中~~活动时...,0,1


In [8]:
import datetime
current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
df.to_csv(f'./out/llm_label_result_{current_time}_{model_name}.csv')